# Update

In [ ]:
import os
import threading
import datetime
import csv
import gspread
from google.oauth2.service_account import Credentials
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import serial
import serial.tools.list_ports
import queue
import time
import re
import cv2

# Column headers for Google Spreadsheet
column_headers = [
    "MM/DD/YYYY hh:mm:ss.SSS", "Temp", "Humidity", "Library_Version", "Session_type",
    "Device_Number", "Battery_Voltage", "Motor_Turns", "FR", "Event", "Active_Poke",
    "Left_Poke_Count", "Right_Poke_Count", "Pellet_Count", "Block_Pellet_Count",
    "Retrieval_Time", "InterPelletInterval", "Poke_Time"
]

# Google Sheets Scope
SCOPE = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]

# Splash Screen Class
class SplashScreen:
    def __init__(self, root, duration=3000):
        self.root = root
        self.root.overrideredirect(True)
        self.root.attributes("-alpha", 1)

        # Resize splash screen for 7-inch screen (800x480 resolution)
        width, height = 800, 480
        self.root.geometry(f"{width}x{height}+0+0")
        self.root.configure(bg="black")

        self.label = tk.Label(self.root, text="McCutcheonLab Technologies", font=("Cascadia Code", 28, "bold"), bg="black", fg="violet")
        self.label.pack(expand=True)
        self.fade_in_out(duration)

    def fade_in_out(self, duration):
        self.fade_in(1000, lambda: self.fade_out(2000, self.close))

    def fade_in(self, time_ms, callback):
        alpha = 0.0
        increment = 1 / (time_ms // 50)
        def fade():
            nonlocal alpha
            if alpha < 1.0:
                alpha += increment
                self.root.attributes("-alpha", alpha)
                self.root.after(50, fade)
            else:
                callback()
        fade()

    def fade_out(self, time_ms, callback):
        alpha = 1.0
        decrement = 1 / (time_ms // 50)
        def fade():
            nonlocal alpha
            if alpha > 0.0:
                alpha -= decrement
                self.root.attributes("-alpha", alpha)
                self.root.after(50, fade)
            else:
                callback()
        fade()

    def close(self):
        self.root.destroy()

# Main GUI Class
class FED3MonitorApp:

    def __init__(self, root):
        self.root = root
        self.root.title("Realtime FED Monitor")
        self.root.geometry("800x480")  # Set for 7-inch touchscreen resolution

        # Variables for the GUI
        self.experimenter_name = tk.StringVar()
        self.experiment_name = tk.StringVar()
        self.json_path = tk.StringVar()
        self.spreadsheet_id = tk.StringVar()
        self.save_path = ""
        self.data_queue = queue.Queue()
        self.serial_ports = {}  # Dictionary mapping port to serial object
        self.threads = []
        self.port_widgets = {}
        self.port_queues = {}
        self.port_threads = {}
        self.log_queue = queue.Queue()
        self.recording_circle = None
        self.recording_label = None
        self.data_to_save = {}
        self.stop_event = threading.Event()
        self.logging_active = False

        self.gspread_client = None

        # For camera handling
        self.port_to_camera_index = {}
        self.camera_objects = {}
        self.recording_states = {}
        self.last_event_times = {}
        self.recording_locks = {}

        self.setup_gui()
        self.update_gui()  # Start the GUI update loop

    def detect_serial_ports(self):
        ports = list(serial.tools.list_ports.comports())
        fed3_ports = []
        for port in ports:
            # Check for VID and PID matching your FED3 devices
            if port.vid == 0x239A and port.pid == 0x800B:
                fed3_ports.append(port.device)
        return fed3_ports

    def detect_cameras(self):
        camera_indices = []
        for index in range(10):  # Adjust range as needed
            cap = cv2.VideoCapture(index)
            if cap.read()[0]:
                camera_indices.append(str(index))
                cap.release()
            else:
                cap.release()
        return camera_indices

    def setup_gui(self):
        # Resize GUI elements to fit 7-inch screen
        self.root.grid_columnconfigure((0, 1, 2, 3), weight=1)

        # Experimenter name input
        tk.Label(self.root, text="Your Name:", font=("Cascadia Code", 8, "bold")).grid(column=0, row=0, sticky=tk.E, padx=2, pady=2)
        self.experimenter_entry = ttk.Entry(self.root, textvariable=self.experimenter_name, width=12)
        self.experimenter_entry.grid(column=1, row=0, sticky=tk.W, padx=2, pady=5)

        # Experiment name input
        tk.Label(self.root, text="Experiment Name:", font=("Cascadia Code", 8, "bold")).grid(column=2, row=0, sticky=tk.E, padx=2, pady=2)
        self.experiment_entry = ttk.Entry(self.root, textvariable=self.experiment_name, width=15)
        self.experiment_entry.grid(column=3, row=0, sticky=tk.E, padx=2, pady=2)

        # JSON file path
        tk.Label(self.root, text="Google API JSON File:", font=("Cascadia Code", 8, "bold")).grid(column=0, row=1, sticky=tk.E, padx=2, pady=2)
        self.json_entry = ttk.Entry(self.root, textvariable=self.json_path, width=25)
        self.json_entry.grid(column=1, row=1, columnspan=2, sticky=tk.W, padx=2, pady=2)
        self.browse_json_button = tk.Button(self.root, text="Browse", command=self.browse_json)
        self.browse_json_button.grid(column=3, row=1, padx=2, pady=2)

        # Google Spreadsheet ID
        tk.Label(self.root, text="Google Spreadsheet ID:", font=("Cascadia Code", 8, "bold")).grid(column=0, row=2, sticky=tk.E, padx=2, pady=2)
        self.spreadsheet_entry = ttk.Entry(self.root, textvariable=self.spreadsheet_id, width=25)
        self.spreadsheet_entry.grid(column=1, row=2, columnspan=2, sticky=tk.W, padx=2, pady=2)

        # Start button
        self.start_button = tk.Button(self.root, text="START", font=("Cascadia Code", 10, "bold"), bg="green", fg="white", command=self.start_logging)
        self.start_button.grid(column=0, row=3, padx=5, pady=5, sticky=tk.W)

        # Stop button
        self.stop_button = tk.Button(self.root, text="STOP(SAVE & QUIT)", font=("Cascadia Code", 10, "bold"), bg="red", fg="white", command=self.stop_logging)
        self.stop_button.grid(column=1, row=3, padx=5, pady=5, sticky=tk.W)

        # Browse button for selecting data folder path
        self.browse_button = tk.Button(self.root, text="Browse Data Folder", font=("Cascadia Code", 10, "bold"), command=self.browse_folder, bg="gold", fg="blue")
        self.browse_button.grid(column=2, row=3, padx=5, pady=5, sticky=tk.W)

        # Canvas for Recording Indicator (reduced size)
        self.canvas = tk.Canvas(self.root, width=100, height=100)
        self.canvas.grid(column=3, row=3, pady=5, sticky=tk.N)

        # Port Status Frame
        self.ports_frame = tk.Frame(self.root)
        self.ports_frame.grid(column=0, row=4, columnspan=4, pady=10)

        # Initialize ports
        detected_ports = self.detect_serial_ports()
        if detected_ports:
            for idx, port in enumerate(detected_ports):
                self.initialize_port_widgets(port, idx)
        else:
            tk.Label(self.ports_frame, text="Connect your FED3 devices!", font=("Cascadia Code", 12), fg="red").pack()

        # Log Section
        log_frame = tk.Frame(self.root)
        log_frame.grid(column=0, row=5, columnspan=4, pady=5, sticky=(tk.N, tk.S, tk.E, tk.W))

        self.log_text = tk.Text(log_frame, height=5, width=100)
        self.log_text.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

        log_scrollbar = ttk.Scrollbar(log_frame, orient="vertical", command=self.log_text.yview)
        log_scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        self.log_text.configure(yscrollcommand=log_scrollbar.set)

        # Handle window close event
        self.root.protocol("WM_DELETE_WINDOW", self.stop_logging)

    def initialize_port_widgets(self, port, idx=None):
        if port in self.port_widgets:
            # Widgets already initialized
            return

        if idx is None:
            idx = len(self.port_widgets)

        port_name = os.path.basename(port)  # Use base name for display
        frame = ttk.LabelFrame(self.ports_frame, text=f"Port {port_name}")
        frame.grid(column=idx % 2, row=idx // 2, padx=5, pady=5, sticky=tk.W)
        status_label = ttk.Label(frame, text="Not Ready", font=("Cascadia Code", 8, "italic"), foreground="red")
        status_label.grid(column=0, row=0, sticky=tk.W)

        # Camera selection dropdown
        camera_label = ttk.Label(frame, text="Camera Index:")
        camera_label.grid(column=0, row=1, sticky=tk.W)
        camera_var = tk.StringVar(value='None')
        camera_indices = self.detect_cameras()
        camera_combobox = ttk.Combobox(frame, textvariable=camera_var, values=['None'] + camera_indices, width=5)
        camera_combobox.grid(column=1, row=1, sticky=tk.W)

        # Test Camera Button
        test_cam_button = tk.Button(frame, text="Test Camera", command=lambda p=port: self.test_camera(p))
        test_cam_button.grid(column=2, row=1, padx=2)

        text_widget = tk.Text(frame, width=35, height=5)
        text_widget.grid(column=0, row=2, columnspan=3, sticky=(tk.N, tk.S, tk.E, tk.W))

        self.port_widgets[port] = {
            'status_label': status_label,
            'text_widget': text_widget,
            'camera_var': camera_var,
            'test_cam_button': test_cam_button,
        }
        self.port_queues[port] = queue.Queue()  # Initialize queue for each port

        # Attempt to open serial port to check readiness
        try:
            ser = serial.Serial(port, 115200, timeout=1)
            ser.close()
            status_label.config(text="Ready", foreground="green")
        except serial.SerialException as e:
            status_label.config(text="Not Ready", foreground="red")
            self.log_queue.put(f"Error with port {port}: {e}")

    def test_camera(self, port):
        camera_index_str = self.port_widgets[port]['camera_var'].get()
        if camera_index_str == 'None':
            messagebox.showinfo("Info", "Please select a camera index to test.")
            return
        camera_index = int(camera_index_str)
        cap = cv2.VideoCapture(camera_index)
        if not cap.isOpened():
            messagebox.showerror("Error", f"Cannot open camera with index {camera_index}")
            return

        cv2.namedWindow(f"Camera {camera_index}", cv2.WINDOW_NORMAL)
        cv2.resizeWindow(f"Camera {camera_index}", 640, 480)
        start_time = time.time()
        while time.time() - start_time < 5:  # Show for 5 seconds
            ret, frame = cap.read()
            if ret:
                cv2.imshow(f"Camera {camera_index}", frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            else:
                messagebox.showerror("Error", f"Failed to read from camera {camera_index}")
                break
        cap.release()
        cv2.destroyWindow(f"Camera {camera_index}")

    def browse_json(self):
        self.json_path.set(filedialog.askopenfilename(title="Select JSON File"))

    def browse_folder(self):
        self.save_path = filedialog.askdirectory(title="Select Folder to Save Data")
        self.log_queue.put(f"Data folder selected: {self.save_path}")
        print(f"Save path selected: {self.save_path}")  # For debugging

    def start_logging(self):
        # Reset stop event and threads list
        self.stop_event.clear()
        self.threads = []
        self.logging_active = True

        # Normalize inputs
        self.experimenter_name.set(self.experimenter_name.get().strip().lower())
        self.experiment_name.set(self.experiment_name.get().strip().lower())
        self.json_path.set(self.json_path.get().strip())
        self.spreadsheet_id.set(self.spreadsheet_id.get().strip())

        # Validate input fields
        if not self.experimenter_name.get() or not self.experiment_name.get():
            messagebox.showerror("Error", "Please provide your name and experiment name.")
            return

        if not self.json_path.get() or not self.spreadsheet_id.get() or not self.save_path:
            messagebox.showerror("Error", "Please provide the JSON file path, Spreadsheet ID, and select a data folder.")
            return

        # Sanitize names to remove invalid characters
        experimenter_name = re.sub(r'[<>:"/\\|?*]', '_', self.experimenter_name.get())
        experiment_name = re.sub(r'[<>:"/\\|?*]', '_', self.experiment_name.get())
        self.experimenter_name.set(experimenter_name)
        self.experiment_name.set(experiment_name)

        try:
            creds = Credentials.from_service_account_file(self.json_path.get(), scopes=SCOPE)
            self.gspread_client = gspread.authorize(creds)
            self.log_queue.put("Connected to Google Sheets!")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to connect to Google Sheets: {e}")
            return

        # Create experiment folder here
        current_time = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
        experimenter_name = self.experimenter_name.get().lower().strip()
        experiment_name = self.experiment_name.get().lower().strip()

        experimenter_folder = os.path.join(self.save_path, experimenter_name)
        experiment_folder = os.path.join(experimenter_folder, f"{experiment_name}_{current_time}")
        self.experiment_folder = experiment_folder  # Store it for use in record_video

        # Create directories with exception handling
        try:
            os.makedirs(experiment_folder, exist_ok=True)
        except Exception as e:
            self.log_queue.put(f"Failed to create directory {experiment_folder}: {e}")
            messagebox.showerror("Error", f"Failed to create directory {experiment_folder}: {e}")
            return

        self.log_queue.put(f"Experiment folder created at {experiment_folder}")

        # Collect camera selections
        for port in self.port_widgets.keys():
            camera_index_str = self.port_widgets[port]['camera_var'].get()
            if camera_index_str == 'None':
                camera_index = None
            else:
                camera_index = int(camera_index_str)
            self.port_to_camera_index[port] = camera_index

        # Initialize cameras
        for port, camera_index in self.port_to_camera_index.items():
            if camera_index is not None:
                cam = cv2.VideoCapture(camera_index)
                if cam.isOpened():
                    self.camera_objects[camera_index] = cam
                    self.log_queue.put(f"Camera {camera_index} associated with port {port}.")
                    # Update status label
                    self.port_widgets[port]['status_label'].config(text="Ready (CAM Ready)", foreground="green")
                else:
                    self.log_queue.put(f"Camera {camera_index} failed to open for port {port}.")
                    self.port_widgets[port]['status_label'].config(text="Ready (CAM Not Connected)", foreground="orange")
                    cam.release()
            else:
                self.log_queue.put(f"No camera associated with port {port}.")
                # Update status label if needed

            # Initialize recording state and lock for this port
            self.recording_states[port] = False
            self.last_event_times[port] = None
            self.recording_locks[port] = threading.Lock()

        # Change recording indicator to ON
        self.show_recording_indicator()

        # Start serial threads
        for port in self.port_widgets.keys():
            self.start_logging_for_port(port)

    def start_logging_for_port(self, port):
        if port in self.port_threads:
            # Already logging for this port
            return
        try:
            ser = serial.Serial(port, 115200, timeout=0.1)  # Set timeout to 0.1 seconds
            self.data_to_save[port] = []
            self.serial_ports[port] = ser  # Store the serial port object
            t = threading.Thread(target=self.read_from_port, args=(ser, f"Port_{os.path.basename(port)}", port))
            t.daemon = True
            t.start()
            self.port_threads[port] = t
            # Update status label if not already set to Ready
            if self.port_widgets[port]['status_label'].cget("text") != "Ready":
                self.port_widgets[port]['status_label'].config(text="Ready", foreground="green")
            self.log_queue.put(f"Started logging from {port}.")
        except serial.SerialException as e:
            self.log_queue.put(f"Error with port {port}: {e}")

    def show_recording_indicator(self):
        if self.recording_circle is None:
            self.recording_circle = self.canvas.create_oval(25, 25, 75, 75, fill="yellow")
        if self.recording_label is None:
            self.recording_label = self.canvas.create_text(50, 90, text="Logging...", font=("Cascadia Code", 12), fill="black")

    def hide_recording_indicator(self):
        if self.recording_circle is not None:
            self.canvas.delete(self.recording_circle)
            self.recording_circle = None
        if self.recording_label is not None:
            self.canvas.delete(self.recording_label)
            self.recording_label = None

    def stop_logging(self):
        # Proceed to close the GUI even if logging hasn't started
        if not self.logging_active:
            self.log_queue.put("Exiting application.")
            self._finalize_exit()
            return

        # Signal threads to stop
        self.stop_event.set()
        self.logging_active = False
        self.log_queue.put("Stopping logging...")

        # Change recording indicator to OFF
        self.hide_recording_indicator()

        # Wait for threads to finish
        for t in self.port_threads.values():
            t.join(timeout=5)  # Set a timeout to prevent hanging
        self.log_queue.put("Logging stopped.")

        # Save data synchronously before destroying the GUI
        self.save_all_data()

        # Release cameras
        for cam in self.camera_objects.values():
            cam.release()
        self.log_queue.put("Cameras released.")

        # Close serial ports
        for ser in self.serial_ports.values():
            try:
                ser.close()
            except Exception:
                pass
        self.log_queue.put("Serial ports closed.")

        # Finalize exit
        self._finalize_exit()

    def save_all_data(self):
        try:
            if not self.save_path:
                self.log_queue.put("Error: Save path is not set.")
                messagebox.showerror("Error", "Save path is not set.")
                return

            self.log_queue.put(f"Saving data to {self.experiment_folder}")

            for port, data_rows in self.data_to_save.items():
                if data_rows:  # Only save if there is data
                    # Use basename of port to avoid slashes in filename
                    port_name = os.path.basename(port)
                    safe_port_name = re.sub(r'[<>:"/\\|?*]', '_', port_name)
                    filename_user = os.path.join(self.experiment_folder, f"{safe_port_name}.csv")
                    try:
                        with open(filename_user, mode='w', newline='') as file:
                            writer = csv.writer(file)
                            writer.writerow(column_headers)
                            writer.writerows(data_rows)
                        self.log_queue.put(f"Data saved for {port} in {filename_user}")
                    except Exception as e:
                        self.log_queue.put(f"Failed to save data for {port}: {e}")
                        messagebox.showerror("Error", f"Failed to save data for {port}: {e}")
                else:
                    self.log_queue.put(f"No data collected from {port}, no file saved.")

        except Exception as e:
            self.log_queue.put(f"Error saving data: {e}")
            messagebox.showerror("Error", f"Failed to save data: {e}")

    def _finalize_exit(self):
        # Inform the user that data has been saved (if logging was active)
        if self.logging_active:
            messagebox.showinfo("Data Saved", "All data has been saved locally.")
        # Quit and destroy the GUI
        self.root.quit()
        self.root.destroy()

    def update_gui(self):
        # Update port text widgets
        for port_identifier, q in self.port_queues.items():
            try:
                while True:
                    message = q.get_nowait()
                    text_widget = self.port_widgets[port_identifier]['text_widget']
                    text_widget.insert(tk.END, message + "\n")
                    text_widget.see(tk.END)
            except queue.Empty:
                pass

        # Update log messages
        try:
            while True:
                log_message = self.log_queue.get_nowait()
                self.log_text.insert(tk.END, f"{datetime.datetime.now()}: {log_message}\n")
                self.log_text.see(tk.END)
        except queue.Empty:
            pass

        # Schedule the next update
        self.root.after(100, self.update_gui)  # Update every 100 ms

    def read_from_port(self, ser, worksheet_name, port_identifier):
        try:
            spreadsheet = self.gspread_client.open_by_key(self.spreadsheet_id.get())
            sheet = self.get_or_create_worksheet(spreadsheet, worksheet_name)
            cached_data = []
            send_interval = 5
            last_send_time = time.time()
            jam_event_occurred = False  # Flag to track JAM events during outage
            device_number = None  # Variable to store Device_Number

            # Get indices for 'Event' and 'Device_Number' columns
            event_index_in_headers = column_headers.index("Event")
            event_index_in_data = event_index_in_headers - 1  # Adjust for timestamp

            device_number_index_in_headers = column_headers.index("Device_Number")
            device_number_index_in_data = device_number_index_in_headers - 1  # Adjust for timestamp

            while not self.stop_event.is_set():
                try:
                    if ser.in_waiting > 0:
                        data = ser.readline().decode('utf-8', errors='replace').strip()
                    else:
                        data = ''
                except serial.SerialException as e:
                    # Device disconnected
                    self.log_queue.put(f"Device on {port_identifier} disconnected: {e}")
                    # Update status label to "Not Ready"
                    if port_identifier in self.port_widgets:
                        self.port_widgets[port_identifier]['status_label'].config(text="Not Ready", foreground="red")
                    break  # Exit the loop and end the thread
                if data:
                    data_list = data.split(",")
                    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
                    data_list = data_list[1:]  # Skip the first item if necessary
                    if len(data_list) == len(column_headers) - 1:
                        row_data = [timestamp] + data_list
                        cached_data.append(row_data)
                        # Instead of updating GUI directly, put message in queue
                        self.port_queues[port_identifier].put(f"Data logged: {data_list}")
                        self.data_to_save[port_identifier].append(row_data)

                        # Extract Device_Number
                        device_number = data_list[device_number_index_in_data].strip()

                        # Check if the event is a JAM event
                        event_value = data_list[event_index_in_data].strip()
                        if event_value == "JAM":
                            jam_event_occurred = True  # Set the flag if JAM occurs

                        # Check if the event is "Pellet"
                        if event_value == "Pellet":
                            # Start video recording
                            with self.recording_locks[port_identifier]:
                                self.last_event_times[port_identifier] = datetime.datetime.now()
                                if not self.recording_states[port_identifier]:
                                    self.recording_states[port_identifier] = True
                                    threading.Thread(target=self.record_video, args=(port_identifier,)).start()
                    else:
                        self.log_queue.put(f"Warning: Data length mismatch on {port_identifier}")
                # Periodically attempt to send cached data
                current_time = time.time()
                if cached_data and (current_time - last_send_time >= send_interval):
                    if not self.stop_event.is_set():
                        try:
                            sheet.append_rows(cached_data)
                            cached_data.clear()

                            # After successful data transmission, check for JAM event
                            if jam_event_occurred:
                                # Create a row with empty strings for all columns
                                jam_row = [''] * len(column_headers)
                                # Set the current timestamp in the first column
                                jam_row[0] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
                                # Place 'JAM' in the 'Event' column
                                jam_row[event_index_in_headers] = "JAM"
                                # Include Device_Number
                                jam_row[device_number_index_in_headers] = device_number
                                # Append the JAM event to Google Sheets
                                sheet.append_row(jam_row)
                                # Log the action
                                self.log_queue.put(f"Additional JAM event logged for {port_identifier} due to prior outage.")
                                # Reset the JAM event flag
                                jam_event_occurred = False
                        except Exception as e:
                            self.log_queue.put(f"Failed to send data to Google Sheets for {port_identifier}: {e}")
                    last_send_time = current_time
                # Sleep briefly to prevent CPU overuse
                time.sleep(0.1)
        except Exception as e:
            self.log_queue.put(f"Error reading from {ser.port}: {e}")
        finally:
            try:
                ser.close()
            except Exception:
                pass
            self.log_queue.put(f"Closed serial port {ser.port}")

    def record_video(self, port_identifier):
        camera_index = self.port_to_camera_index.get(port_identifier)
        if camera_index is None:
            self.log_queue.put(f"No camera associated with port {port_identifier}")
            return

        cam = self.camera_objects.get(camera_index)
        if cam is None:
            self.log_queue.put(f"Camera {camera_index} not available for port {port_identifier}")
            return

        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        port_name = os.path.basename(port_identifier)
        safe_port_name = re.sub(r'[<>:"/\\|?*]', '_', port_name)

        path = os.path.join(self.experiment_folder, safe_port_name)
        os.makedirs(path, exist_ok=True)
        filename = os.path.join(path, f"{safe_port_name}_camera_{timestamp}.avi")

        frame_width = int(cam.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))
        if frame_width == 0 or frame_height == 0:
            self.log_queue.put(f"Camera {camera_index} is not returning frames.")
            with self.recording_locks[port_identifier]:
                self.recording_states[port_identifier] = False
                self.last_event_times[port_identifier] = None
            return
        out = cv2.VideoWriter(filename, cv2.VideoWriter_fourcc(*'XVID'), 20.0, (frame_width, frame_height))

        try:
            while True:
                with self.recording_locks[port_identifier]:
                    last_event_time = self.last_event_times[port_identifier]
                if last_event_time is None:
                    break
                time_since_last_event = (datetime.datetime.now() - last_event_time).total_seconds()
                if time_since_last_event > 30:
                    break  # No 'Pellet' event within the last 30 seconds, stop recording
                # Else, continue recording
                ret, frame = cam.read()
                if ret:
                    out.write(frame)
                else:
                    self.log_queue.put(f"Failed to read frame from camera {camera_index}")
                    break
                time.sleep(0.05)  # Adjust as needed for frame rate
        finally:
            out.release()
            with self.recording_locks[port_identifier]:
                self.recording_states[port_identifier] = False
                self.last_event_times[port_identifier] = None
            self.log_queue.put(f"Video saved as {filename}")

    def get_or_create_worksheet(self, spreadsheet, title):
        try:
            return spreadsheet.worksheet(title)
        except gspread.exceptions.WorksheetNotFound:
            sheet = spreadsheet.add_worksheet(title=title, rows="1000", cols="20")
            sheet.append_row(column_headers)
            return sheet

# Main execution
if __name__ == "__main__":
    # Splash screen
    splash_root = tk.Tk()
    splash_screen = SplashScreen(splash_root)
    splash_root.after(3000, splash_screen.close)
    splash_root.mainloop()

    # Main GUI
    root = tk.Tk()
    app = FED3MonitorApp(root)
    root.mainloop()
